In [488]:
import torch
import numpy as np 
import csv
import pandas as pd
 
def read_csv(file_path):
    data = pd.read_csv(file_path)
    return data

data = read_csv("C:\\Users\\Administrator\\Desktop\\Russia_losses_data.csv")
print(data)

          date  personnel_ac  personnel_daily  aircraft  helicopter  tank  \
0    2022/2/25          2800             2800        10           7    80   
1    2022/2/26          4300             1500        27          26   146   
2    2022/2/27          4500              200        27          26   150   
3    2022/2/28          5300              800        29          29   150   
4     2022/3/1          5710              410        29          29   198   
..         ...           ...              ...       ...         ...   ...   
831   2024/6/5        513700             1280       357         326  7806   
832   2024/6/6        515000             1300       357         326  7828   
833   2024/6/7        516080             1080       357         326  7834   
834   2024/6/8        517290             1210       357         326  7843   
835   2024/6/9        518560             1270       357         326  7869   

       APC  field artillery   MRL  drone  ...  special equipment  \
0      

In [489]:
# 数据预处理
data = data.iloc[30:, :]
# 划分训练集、测试集
train = data.iloc[0:-60, :]
test = data.iloc[-60:, :]

X_train = train.iloc[:, 12:]
Y_train = train.iloc[:, 2]

X_test = test.iloc[:, 12:]
Y_test = test.iloc[:, 2]

print(X_test.shape)
print(Y_test.shape)
print(X_train.shape)
print(Y_train.shape)

(60, 9)
(60,)
(746, 9)
(746,)


In [490]:
# 定义线性回归模型
class LinearRegression(torch.nn.Module):
    def __init__(self, n_feature, n_output):
        super(LinearRegression, self).__init__()
        self.pred = torch.nn.Linear(n_feature, n_output)
    
    def forward(self, x):
        out = self.pred(x)
        return out

linear_regression = LinearRegression(9, 1)
print(linear_regression)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(linear_regression.parameters(), lr=0.001)

LinearRegression(
  (pred): Linear(in_features=9, out_features=1, bias=True)
)


In [491]:
# 模型训练
for i in range(10000):
    x = torch.tensor(X_train.values, dtype=torch.float32)
    y = torch.tensor(Y_train.values, dtype=torch.float32)
    pred = linear_regression.forward(x)
    pred = torch.squeeze(pred)
    loss_train = loss_func(pred, y) * 0.001
    loss_train.backward()
    optimizer.step()
    optimizer.zero_grad()
    print('ite:{}, loss:{}'.format(i, loss_train))
    

ite:0, loss:428.90594482421875
ite:1, loss:428.8283386230469
ite:2, loss:428.75067138671875
ite:3, loss:428.673095703125
ite:4, loss:428.595458984375
ite:5, loss:428.51788330078125
ite:6, loss:428.4403076171875
ite:7, loss:428.3627014160156
ite:8, loss:428.28515625
ite:9, loss:428.20758056640625
ite:10, loss:428.1299743652344
ite:11, loss:428.0525207519531
ite:12, loss:427.97491455078125
ite:13, loss:427.8974304199219
ite:14, loss:427.8199462890625
ite:15, loss:427.74249267578125
ite:16, loss:427.6649475097656
ite:17, loss:427.5874938964844
ite:18, loss:427.510009765625
ite:19, loss:427.43255615234375
ite:20, loss:427.35516357421875
ite:21, loss:427.2777404785156
ite:22, loss:427.20025634765625
ite:23, loss:427.1229248046875
ite:24, loss:427.0455322265625
ite:25, loss:426.9681091308594
ite:26, loss:426.89080810546875
ite:27, loss:426.8133850097656
ite:28, loss:426.7360534667969
ite:29, loss:426.6587829589844
ite:30, loss:426.5814208984375
ite:31, loss:426.5041198730469
ite:32, loss:426

In [492]:
# 测试线性回归模型
x = torch.tensor(X_test.values, dtype=torch.float32)
y = torch.tensor(Y_test.values, dtype=torch.float32)
pred = linear_regression.forward(x)
pred = torch.squeeze(pred)
print(pred.data)
loss_train = loss_func(pred, y) * 0.001
print(loss_train)

tensor([ 823.7341,  989.0239, 1181.2166,  835.3412,  877.6397,  392.2552,
         331.9463,  277.6933,  581.5353,  650.5865,  992.6210,  367.6374,
         559.2368,  769.6929,  544.9233,  651.6717,  694.9507,  790.2941,
         663.1533,  763.4713,  393.5927,  796.5742, 1087.0057,  994.0374,
         981.4459, 1013.5314,  944.5460,  598.8770,  978.6628, 1113.8042,
        1281.0457,  808.8323,  987.9807, 1076.3220,  961.9849,  717.3426,
         987.8173, 1051.5640, 1181.5444, 1430.0629, 1188.0435, 1010.1937,
         998.9365,  820.9001,  569.1955,  584.7371,  799.6686, 1024.8705,
        1016.3884,  802.6266, 1137.4937,  913.8913, 1593.9257, 1004.3755,
        1101.7816,  938.2955, 1268.0729, 1086.2109,  959.1754, 1231.3481])
tensor(128.0033, grad_fn=<MulBackward0>)


In [493]:
# 定义两层神经网络
class NN(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(NN, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.pred = torch.nn.Linear(n_hidden, n_output)
    
    def forward(self, x):
        h1 = self.hidden(x)
        a1 = torch.nn.functional.relu(h1)
        out = self.pred(a1)
        a2 = torch.nn.functional.relu(out)
        return a2

nn = NN(9, 50, 1)
print(nn)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(nn.parameters(), lr=0.001)

NN(
  (hidden): Linear(in_features=9, out_features=50, bias=True)
  (pred): Linear(in_features=50, out_features=1, bias=True)
)


In [494]:
# 模型训练
for i in range(1000):
    x = torch.tensor(X_train.values, dtype=torch.float32)
    y = torch.tensor(Y_train.values, dtype=torch.float32)
    pred = nn.forward(x)
    pred = torch.squeeze(pred)
    loss_train = loss_func(pred, y) * 0.001
    loss_train.backward()
    optimizer.step()
    optimizer.zero_grad()
    print('ite:{}, loss:{}'.format(i, loss_train))

ite:0, loss:417.4604187011719
ite:1, loss:417.2232360839844
ite:2, loss:416.98577880859375
ite:3, loss:416.7479553222656
ite:4, loss:416.5096130371094
ite:5, loss:416.2708740234375
ite:6, loss:416.0317077636719
ite:7, loss:415.7919006347656
ite:8, loss:415.55157470703125
ite:9, loss:415.3106689453125
ite:10, loss:415.06903076171875
ite:11, loss:414.82647705078125
ite:12, loss:414.5826110839844
ite:13, loss:414.337646484375
ite:14, loss:414.0913391113281
ite:15, loss:413.8435974121094
ite:16, loss:413.5945129394531
ite:17, loss:413.34393310546875
ite:18, loss:413.091552734375
ite:19, loss:412.8371887207031
ite:20, loss:412.58087158203125
ite:21, loss:412.32257080078125
ite:22, loss:412.062255859375
ite:23, loss:411.7994079589844
ite:24, loss:411.5340576171875
ite:25, loss:411.265869140625
ite:26, loss:410.99456787109375
ite:27, loss:410.7203369140625
ite:28, loss:410.44287109375
ite:29, loss:410.161865234375
ite:30, loss:409.8770446777344
ite:31, loss:409.588623046875
ite:32, loss:409.2

In [495]:
# 测试神经网络
x = torch.tensor(X_test.values, dtype=torch.float32)
y = torch.tensor(Y_test.values, dtype=torch.float32)
pred = nn.forward(x)
pred = torch.squeeze(pred)
print(pred.data)
loss_train = loss_func(pred, y) * 0.001
print(loss_train)

tensor([ 896.9355, 1021.3517, 1254.5377,  916.1860,  934.3377,  426.5377,
         377.1888,  326.7662,  603.2501,  671.9001, 1078.9220,  443.0319,
         599.8568,  842.2896,  615.7820,  775.8521,  767.7654,  870.1847,
         726.2800,  861.5980,  435.3516,  877.6746, 1221.3881, 1092.9110,
        1071.3480, 1057.1327, 1048.8093,  670.8096, 1019.9664, 1161.5448,
        1390.3751,  822.7999, 1068.8622, 1105.4166, 1023.6826,  788.5684,
        1045.2811, 1114.9688, 1244.8684, 1438.6488, 1246.9630, 1042.7679,
        1041.5428,  931.9650,  643.4480,  662.0589,  845.0421, 1144.9471,
        1152.9603,  904.6764, 1215.6329, 1047.3585, 1680.2255, 1076.9930,
        1256.7379, 1032.1515, 1372.2375, 1178.0500, 1006.4513, 1354.1074])
tensor(98.3519, grad_fn=<MulBackward0>)


In [496]:
# 定义循环神经网络
class RNN(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(RNN, self).__init__()
        self.rnn = torch.nn.RNN(
            input_size=n_feature,
            hidden_size=n_hidden,
            batch_first=True
        )
        self.pred = torch.nn.Linear(n_hidden, n_output)
    
    def forward(self, x, h_stage):
        r_out, h_stage = self.rnn(x, h_stage)
        outs=[]
        for time in range(r_out.size(1)):
            outs.append(self.pred(r_out[:, time, :]))
        return torch.stack(outs, dim=-1), h_stage

rnn = RNN(9, 50, 1)
print(rnn)
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1)

RNN(
  (rnn): RNN(9, 50, batch_first=True)
  (pred): Linear(in_features=50, out_features=1, bias=True)
)


In [497]:
# 模型训练
h_stage = None
for i in range(100):
    x = torch.tensor(X_train.values, dtype=torch.float32)
    x = torch.unsqueeze(x, 0)

    y = torch.tensor(Y_train.values, dtype=torch.float32)
    y = torch.unsqueeze(y, 0)

    pred, h_stage = rnn(x, h_stage)
    h_stage = h_stage.data
    pred = torch.squeeze(pred, 0)
    loss_train = loss_func(pred, y) * 0.001
    optimizer.zero_grad()
    loss_train.backward(retain_graph = True)
    optimizer.step()
    print('ite:{}, loss:{}'.format(i, loss_train))

ite:0, loss:420.84765625
ite:1, loss:416.4404602050781
ite:2, loss:412.4744873046875
ite:3, loss:408.13421630859375
ite:4, loss:403.612548828125
ite:5, loss:398.7735900878906
ite:6, loss:393.7493896484375
ite:7, loss:388.6295166015625
ite:8, loss:383.4346008300781
ite:9, loss:378.2044372558594
ite:10, loss:372.968994140625
ite:11, loss:367.740478515625
ite:12, loss:362.5287170410156
ite:13, loss:357.3426513671875
ite:14, loss:352.189453125
ite:15, loss:347.0747375488281
ite:16, loss:342.0031433105469
ite:17, loss:336.9784240722656
ite:18, loss:332.003662109375
ite:19, loss:327.08154296875
ite:20, loss:322.2141418457031
ite:21, loss:317.4033508300781
ite:22, loss:312.6507263183594
ite:23, loss:307.9574279785156
ite:24, loss:303.32464599609375
ite:25, loss:298.7532043457031
ite:26, loss:294.2438659667969
ite:27, loss:289.7971496582031
ite:28, loss:285.41357421875
ite:29, loss:281.09344482421875
ite:30, loss:276.8370361328125
ite:31, loss:272.64447021484375
ite:32, loss:268.5159606933594


In [498]:
# 测试神经网络
x = torch.tensor(X_test.values, dtype=torch.float32)
x = torch.unsqueeze(x, 0)
y = torch.tensor(Y_test.values, dtype=torch.float32)
y = torch.unsqueeze(y, 0)
h_stage = None
pred, h_stage = rnn(x, h_stage)
pred = torch.squeeze(pred, 0)
print(pred.data)
loss_train = loss_func(pred, y) * 0.001
print(loss_train)

tensor([[407.3857, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619,
         407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618,
         407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619,
         407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618,
         407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619,
         407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618,
         407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619,
         407.8618, 407.8619, 407.8618, 407.8619, 407.8618, 407.8619, 407.8618,
         407.8619, 407.8618, 407.8619, 407.8618]])
tensor(587.2416, grad_fn=<MulBackward0>)
